Тест выполнил: Колотилин Валентин
контактный адрес электронной почты: kolotilinvalentin@gmail.com 
telegram: @vdkolotilin

In [118]:
#Загрузка необходимых библиотек
import pandas as pd
import plotly.express as px

In [119]:
#зашрузка данных с CSV файла
df = pd.read_csv('loan_data.csv')

# 0 - Обзор данных

## 0.1 - Краткий обзор данных ("по верхам")

In [120]:
#Посмотрим на первые 5 строк набора данных
df.head(5)

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
0,12436,1,2023-02-20T03:00:00,paid,532362.0,2023-01-04T15:22:07.106088,10,active,no restruct,No auto-approve,4. platinum
1,12436,2,2023-03-20T03:00:00,paid,521224.0,2023-01-04T15:22:07.106088,10,active,no restruct,No auto-approve,4. platinum
2,12436,3,2023-04-20T03:00:00,paid,522177.0,2023-01-04T15:22:07.106088,10,active,no restruct,No auto-approve,4. platinum
3,12436,4,2023-05-20T03:00:00,paid,521007.0,2023-01-04T15:22:07.106088,10,active,no restruct,No auto-approve,4. platinum
4,12436,5,2023-06-20T03:00:00,paid,520816.0,2023-01-04T15:22:07.106088,10,active,no restruct,No auto-approve,4. platinum


In [121]:
# Посмотрим на краткую описательную статистику набора данных
df.describe(include='all')

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
count,71025.000000,71025.000000,70981,71025,7.102500e+04,71025,71025.000000,71025,71025,71025,71025
unique,NaN,NaN,1420,6,NaN,4712,NaN,4,2,2,5
top,NaN,NaN,2024-02-29T03:00:00,unscheduled,NaN,2023-02-15T17:16:09.063367,NaN,active,no restruct,No auto-approve,2. silver
freq,NaN,NaN,564,36572,NaN,59,NaN,59885,66009,67890,46993
mean,15500.029145,10.432059,NaN,NaN,1.823901e+05,NaN,13.089335,NaN,NaN,NaN,NaN
std,1700.309092,7.155209,NaN,NaN,2.479868e+05,NaN,3.567788,NaN,NaN,NaN,NaN
min,12436.000000,1.000000,NaN,NaN,0.000000e+00,NaN,1.000000,NaN,NaN,NaN,NaN
25%,14031.000000,4.000000,NaN,NaN,5.956500e+04,NaN,11.000000,NaN,NaN,NaN,NaN
50%,15588.000000,9.000000,NaN,NaN,1.184790e+05,NaN,13.000000,NaN,NaN,NaN,NaN
75%,16944.000000,15.000000,NaN,NaN,2.349670e+05,NaN,16.000000,NaN,NaN,NaN,NaN


In [122]:
# Посмотрим количество уникальных значений в каждом столбце
df.nunique()

id                       4712
number                     59
payment_date             1420
status                      6
amount                  66880
start_date               4712
loan_rating_int            18
loan_status_str             4
restruct_status             2
auto_approval               2
borrower_issue_class        5
dtype: int64

In [123]:
# Посмотрим количество нулевых значений в каждом столбце
df.isnull().sum()

id                       0
number                   0
payment_date            44
status                   0
amount                   0
start_date               0
loan_rating_int          0
loan_status_str          0
restruct_status          0
auto_approval            0
borrower_issue_class     0
dtype: int64

## 0.2 - Детальный обзор данных в каждом столбце


#### 1) ID - идентификатор займа.
Поскольку всего строк более 70 000 тысяч, а уникальных значений ID только 4712, можно сделать вывод, что ID не является уникальных значением каждой отдельной "транзакции", а относится к другому показателю - заемщику, типа займа или пр.

#### 2) Number - порядковый номер платежа.
уникальных значений number 59. Проверим, идут ли они друг за другом без пропусков

In [124]:
print(df['number'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59]


Да, идут. Можем сделать вывод, что внутри каждого ID есть от 1 до 59 отдельных платежей. Таким образом, ID всё же является уникальным идентификатором займа, и в нем может быть более 1 транзакции

#### 3)  payment_date - Дата платежа.
Дата каждого отдельного платежа (по Number в составе каждого ID)
(просмотрев первые 5 строк набора данных видно, что дата не является одинаковой для всех значений number в составе каждого ID, значит платежи по number могут проходить в разное время (не одновременно для всех number в составе каждого ID)

#### 4) Status - Статус платежа.

In [125]:
# Просмотрим варианты значений в столбце статус
print(df['status'].unique())

['paid' 'next' 'unscheduled' 'overdue' 'potential default' 'defaulted']


варианты статуса платежа - «оплачен», «следующий», «внеплановый», «просрочен», «потенциальный дефолт», «дефолт»
("Дефолт" в контексте финансов и кредитования означает невыполнение заемщиком своих обязательств по возврату займа или кредита в установленные сроки)

#### 5) amount - сумма платежа.
сумма платежа по каждому платежу number в составе ID

In [126]:
# Посмотрим на боксплот по сумме платежей
fig = px.box(df, y='amount', title='Boxplot of Amount')
fig.show()

Видим много выбросов на графике.
Поскольку:
1) на данном этапе мы не можем объяснить причину этих выбросов;
2) действительно могут быть большие платежи;
Мы не будем удалять данные выбросы из набора данных

In [127]:
df['amount'].dtype

dtype('float64')

#### 6) start_date - Дата взятия займа.
количество уникальных значений в start_date = количеству ID. Можем предположить, что это дата создания блока транзакций по каждому отдельному ID

Мы видим, что в столбце start_date после даты и времени есть набор числовых символов. Проверим, каким типом данныз являеются значения столбца start_date

In [128]:
df['start_date'].dtype

dtype('O')

такой тип данных указывает на то, что столбец start_date содержит строки. далее нужно будет обработать эти данные для получения даных типа "datetime"

#### 7) loan_rating_int - Рейтинг займа.

In [129]:
sorted(df['loan_rating_int'].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

loan_rating_int принимает значения от 1 до 18. Вероятно, чем выше показатель - тем надежнее займ

#### 8) loan_status_str - Статус займа.

In [130]:
df['loan_status_str'].unique()

array(['active', 'closed', 'delayed', 'default'], dtype=object)

#### 9) restruct_status -  Статус реструктуризации.

In [131]:
df['restruct_status'].unique()

array(['no restruct', 'restruct'], dtype=object)

Вероятно, данный показатель указывает, подвергался ли кредит какой-либо реструктуризации или изменениям его первоначальных условий.

#### 10) auto_approval -  Был ли займ одобрен автоматически.

In [132]:
df['auto_approval'].unique()

array(['No auto-approve', 'Auto-approve'], dtype=object)

#### 11) borrower_issue_class - Класс заемщика на момент взятия займа

In [133]:
df['borrower_issue_class'].unique()

array(['4. platinum', '2. silver', '1. bronze', '3. gold', '5. diamond'],
      dtype=object)

Возможно, в будущем нужно будет либо оставить только числа, либо только названия для удобства аналитики

## 0.3 - Обработка данных
### перед началом работы с набором данных проверим, нужно ли сделать какие-либо правки

In [134]:
# столбец payment_date имеет 44 пропуска. Посмотрим, какого размера платежи были в данные даты
df_missing_payment_date = df[df['payment_date'].isna()]
df_missing_payment_date

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
1070,12566,7,NaN,paid,3158.210,2023-01-17T08:36:05.306626,16,closed,no restruct,No auto-approve,2. silver
2745,12713,13,NaN,paid,705.333,2023-02-02T20:37:34.901188,14,closed,no restruct,No auto-approve,2. silver
3227,12751,9,NaN,paid,373.116,2023-02-01T05:09:56.449434,17,closed,no restruct,No auto-approve,3. gold
4076,12820,7,NaN,potential default,148505.000,2023-02-08T19:44:17.51937,18,default,no restruct,No auto-approve,2. silver
4760,12878,13,NaN,paid,872.680,2023-02-14T10:29:25.249524,16,closed,no restruct,No auto-approve,2. silver
5785,12973,11,NaN,paid,4602.920,2023-02-21T10:55:16.721167,15,closed,no restruct,No auto-approve,2. silver
5938,12986,9,NaN,paid,5224.080,2023-02-23T17:19:33.111559,14,closed,no restruct,No auto-approve,2. silver
6033,12994,13,NaN,paid,3296.010,2023-02-27T16:10:00.355529,16,closed,no restruct,No auto-approve,2. silver
6071,12997,2,NaN,paid,9927.180,2023-02-28T13:08:05.109576,18,closed,no restruct,No auto-approve,2. silver
6584,13038,13,NaN,paid,5044.220,2023-02-28T14:27:49.993586,18,closed,no restruct,No auto-approve,1. bronze


In [135]:
df_missing_payment_date.describe(include='all')

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
count,44.000000,44.000000,0,44,44.000000,44,44.000000,44,44,44,44
unique,NaN,NaN,0,3,NaN,44,NaN,4,1,2,4
top,NaN,NaN,NaN,paid,NaN,2023-01-17T08:36:05.306626,NaN,closed,no restruct,No auto-approve,2. silver
freq,NaN,NaN,NaN,34,NaN,1,NaN,34,44,42,25
mean,14445.090909,7.022727,NaN,NaN,23192.280795,NaN,14.886364,NaN,NaN,NaN,NaN
std,1316.074215,2.945147,NaN,NaN,70390.943098,NaN,2.364622,NaN,NaN,NaN,NaN
min,12566.000000,2.000000,NaN,NaN,0.000000,NaN,8.000000,NaN,NaN,NaN,NaN
25%,13116.000000,5.000000,NaN,NaN,3150.922500,NaN,13.000000,NaN,NaN,NaN,NaN
50%,14399.000000,7.000000,NaN,NaN,6265.290000,NaN,15.000000,NaN,NaN,NaN,NaN
75%,15344.250000,7.250000,NaN,NaN,16484.200000,NaN,17.000000,NaN,NaN,NaN,NaN


In [136]:
# Поскольку:
# 1) у нас не может быть среднего значения даты для заполнния пропусков в наборе данных
# 2) Количество пропусков мало в сравнении с общим количеством строк (44 и 71 025)
# 3) В строках с пропусками нет аномально больших значений amount 

#данные строки для удобства аналитики можем удалить

df = df.dropna(subset=['payment_date'])
df.describe()

,id,number,amount,loan_rating_int
count,70981.000000,70981.000000,7.098100e+04,70981.000000
mean,15500.683084,10.434173,1.824887e+05,13.088221
std,1700.324546,7.156555,2.480259e+05,3.568138
min,12436.000000,1.000000,1.120000e+02,1.000000
25%,14031.000000,4.000000,5.965370e+04,11.000000
50%,15591.000000,9.000000,1.185630e+05,13.000000
75%,16944.000000,15.000000,2.350380e+05,16.000000
max,19409.000000,59.000000,1.008850e+07,18.000000


In [137]:
# Также нужно преобразовать значения в столбце start_date в тип данных "Datetime"

df.loc[:, 'start_date'] = pd.to_datetime(df['start_date'])
df['start_date'].dtype

dtype('<M8[ns]')

Такой тип данных считывает строки после времени как микросекунды.

# 1 - Проведите разведочный анализ займов в разрезе рейтингов

Нужно провести первичный анализ данных по столбцу рейтинг (loan_rating_int). Для этого мы можем построить следующие графики для визуализации:

1. количество займов (id) по рейтингу (loan_rating_int) в разрезе статуса выплат (status)
2. сумма выплат (amount) по рейтингу (loan_rating_int) в разрезе статуса выплат (status)

### 1.1 - количество займов (id) по рейтингу (loan_rating_int) в разрезе статуса выплат (status)

In [138]:
# Группировка данных по рейтингу и статусу выплат, подсчет количества займов
loan_counts = df.groupby(['loan_rating_int', 'status']).size().reset_index(name='counts')
loan_counts = loan_counts.sort_values(by=['loan_rating_int', 'status'])

# Построение графика
fig = px.bar(loan_counts, 
             x='loan_rating_int', 
             y='counts', 
             color='status', 
             title='Количество займов по рейтингу в разрезе статуса выплат',
             labels={'loan_rating_int': 'Рейтинг займа', 'counts': 'Количество займов', 'status': 'Статус выплат'})
fig.update_xaxes(title='Рейтинг займа')
fig.update_yaxes(title='Количество займов, шт.')
fig.show()

#### Выводы:
1. большинство займов приходится на статусы выплат "paid" и "unscheduled"
2. с увеличением рейтинга займа растёт доля займов со статусом "defaulted" и "potentially defaulted"

### 1.2 - сумма выплат (amount) по рейтингу (loan_rating_int) в разрезе статуса выплат (status)

In [139]:
# Группировка данных по рейтингу, статусу выплат и сумме выплат
loan_amount = df.groupby(['loan_rating_int', 'status'])['amount'].sum().reset_index()
loan_amount = loan_amount.sort_values(by=['loan_rating_int', 'status'])

# Построение графика
fig = px.bar(loan_amount, 
             x='loan_rating_int', 
             y='amount', 
             color='status', 
             title='Сумма выплат по рейтингу в разрезе статуса выплат',
             labels={'loan_rating_int': 'Рейтинг займа', 'amount': 'Сумма выплат', 'status': 'Статус выплат'})
fig.update_xaxes(title='Рейтинг займа')
fig.update_yaxes(title='Сумма выплат, руб.')
fig.show()

#### Выводы аналогичны пункту 1.1

# 2 - Проиллюстрируйте как менялось отношение Non-performing займов к общей выдаче по полю amount до сегодняшнего дня
#### - К NPL платежам будем относить overdue_statuses = ['overdue','potential default', 'defaulted']
#### - Будем считать, что займы становятся просроченными безвозвратно.
#### - Будем считать, что в момент возникновения просроченного платежа в NPL уходит вся оставшаяся сумма займа.  


NPL - Non Performing Loans - непроизводительные кредиты - кредиты, которые стали просроченными и по которым заемщик не выплачивает проценты или основной долг в установленные сроки.

In [140]:
# Создадим новый набор данных только с NPL займами
overdue_statuses = ['overdue', 'potential default', 'defaulted']
df_npl = df[df['status'].isin(overdue_statuses)]
df_npl

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
169,12495,13,2024-02-15T03:00:00,overdue,64461.6,2023-01-09 07:23:00.226095,13,delayed,no restruct,No auto-approve,2. silver
171,12495,15,2024-04-15T03:00:00,potential default,56989.4,2023-01-09 07:23:00.226095,13,delayed,no restruct,No auto-approve,2. silver
172,12495,16,2024-05-15T03:00:00,potential default,56824.9,2023-01-09 07:23:00.226095,13,delayed,no restruct,No auto-approve,2. silver
173,12495,17,2024-06-15T03:00:00,potential default,56743.6,2023-01-09 07:23:00.226095,13,delayed,no restruct,No auto-approve,2. silver
174,12495,18,2024-07-15T03:00:00,potential default,56583.0,2023-01-09 07:23:00.226095,13,delayed,no restruct,No auto-approve,2. silver
...,...,...,...,...,...,...,...,...,...,...,...
70925,19088,11,2024-12-05T03:00:00,potential default,193199.0,2023-11-10 15:28:38.994720,15,delayed,restruct,No auto-approve,2. silver
70926,19088,12,2025-01-05T03:00:00,potential default,192746.0,2023-11-10 15:28:38.994720,15,delayed,restruct,No auto-approve,2. silver
70927,19088,13,2025-02-05T03:00:00,potential default,192198.0,2023-11-10 15:28:38.994720,15,delayed,restruct,No auto-approve,2. silver
70928,19088,14,2025-03-05T03:00:00,potential default,191539.0,2023-11-10 15:28:38.994720,15,delayed,restruct,No auto-approve,2. silver


In [141]:
# Посчитаем общую сумму amount на каждую дату
df_grouped_by_date = df.groupby(df['start_date'].dt.date)
total_amount = df_grouped_by_date['amount'].sum()
total_amount

start_date
2023-01-01    6.959818e+06
2023-01-04    1.393893e+07
2023-01-05    1.473661e+06
2023-01-08    1.107983e+06
2023-01-09    4.118095e+07
                  ...     
2023-12-27    1.915303e+08
2023-12-28    1.858212e+08
2023-12-29    1.556405e+08
2023-12-30    3.912902e+07
2023-12-31    6.034243e+05
Name: amount, Length: 340, dtype: float64

In [142]:
# Посчитаем сумму amount по фильтрам overdue_statuses на каждую дату
df_npl_grouped_by_date = df_npl.groupby(df['start_date'].dt.date)
npl_amount = df_npl_grouped_by_date['amount'].sum()
npl_amount

start_date
2023-01-05      386922.80
2023-01-09     5990422.25
2023-01-11     6977412.90
2023-01-12     7104941.90
2023-01-13      777444.25
                 ...     
2023-12-25     7661080.00
2023-12-26     2667906.60
2023-12-27    10041919.00
2023-12-28     3104457.70
2023-12-29     2168006.00
Name: amount, Length: 166, dtype: float64

In [143]:
# в npl в 2 раза меньше значений, поэтому нужно либо
# 1) заполнить 0 пропуски 2) удалить строки из бОльшего набора данных

In [144]:
type(npl_amount)

pandas.core.series.Series

In [145]:
# объединим 2 получившихся набора
new_df = pd.merge(npl_amount, total_amount, on='start_date')
new_df

,amount_x,amount_y
start_date,,
2023-01-05,386922.80,1.473661e+06
2023-01-09,5990422.25,4.118095e+07
2023-01-11,6977412.90,1.949228e+07
2023-01-12,7104941.90,2.044517e+07
2023-01-13,777444.25,2.267739e+07
...,...,...
2023-12-25,7661080.00,1.903708e+08
2023-12-26,2667906.60,9.865354e+07
2023-12-27,10041919.00,1.915303e+08


In [146]:
# пропуски просто не замэтчились

In [147]:
# добавим новый столбец и построим по нему график
new_df['npl/total']=new_df['amount_x']/new_df['amount_y']
new_df

,amount_x,amount_y,npl/total
start_date,,,
2023-01-05,386922.80,1.473661e+06,0.262559
2023-01-09,5990422.25,4.118095e+07,0.145466
2023-01-11,6977412.90,1.949228e+07,0.357958
2023-01-12,7104941.90,2.044517e+07,0.347512
2023-01-13,777444.25,2.267739e+07,0.034283
...,...,...,...
2023-12-25,7661080.00,1.903708e+08,0.040243
2023-12-26,2667906.60,9.865354e+07,0.027043
2023-12-27,10041919.00,1.915303e+08,0.052430


In [148]:
#построим по нему график
fig = px.line(new_df,
              x=new_df.index,
              y='npl/total',
              title='Отоншение непроизводительных займов (NPL) ко всем займам в 2023 г. ')

fig.update_xaxes(title_text='Дата')
fig.update_yaxes(title_text='NPL / Total Amount')

fig.show()

# 3 - * Проведите когортный анализ дефолтности в разрезе месяца выдачи займа 
#### - Ушедшими в дефолт будем считать платежи в статусе status == "defaulted"
#### - Сумму дефолта по когорте считать накопленным итогом от месяца к месяцу

Когортный анализ — это метод анализа данных, который позволяет изучать поведение группы пользователей (или клиентов) во времени. Группы, которые анализируются, называются когортами. Когорты обычно формируются на основе определенного события или характеристики, которое произошло в определенный период времени.

Основные этапы когортного анализа:

Выбор когорты: Определение критериев для формирования когорт (например, месяц регистрации пользователя).

Группировка данных: Разбиение пользователей на когорты и группировка данных по времени (например, по месяцам).

Анализ поведения: Изучение изменений в поведении когорт с течением времени, таких как уровень удержания, конверсия или средний чек.

Визуализация результатов: Построение графиков и диаграмм для наглядного представления изменений в поведении когорт.

In [149]:
df3=df[df['status'] == 'defaulted']
df3

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class
351,12506,11,2023-12-07T03:00:00,defaulted,266386.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver
352,12506,12,2024-01-07T03:00:00,defaulted,126625.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver
353,12506,13,2024-02-07T03:00:00,defaulted,126418.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver
354,12506,14,2024-03-07T03:00:00,defaulted,126044.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver
355,12506,15,2024-04-07T03:00:00,defaulted,125990.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver
...,...,...,...,...,...,...,...,...,...,...,...
67150,18046,22,2025-07-01T03:00:00,defaulted,36033.2,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver
67151,18046,23,2025-08-01T03:00:00,defaulted,35977.0,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver
67152,18046,24,2025-09-01T03:00:00,defaulted,35909.7,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver
67153,18046,25,2025-10-01T03:00:00,defaulted,35837.8,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver


In [150]:
# Укажем начало месяца из даты start_date
df3 = df3.copy()
df3['new_month'] = df3['start_date'].dt.to_period('M')
df3

,id,number,payment_date,status,amount,start_date,loan_rating_int,loan_status_str,restruct_status,auto_approval,borrower_issue_class,new_month
351,12506,11,2023-12-07T03:00:00,defaulted,266386.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver,2023-01
352,12506,12,2024-01-07T03:00:00,defaulted,126625.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver,2023-01
353,12506,13,2024-02-07T03:00:00,defaulted,126418.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver,2023-01
354,12506,14,2024-03-07T03:00:00,defaulted,126044.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver,2023-01
355,12506,15,2024-04-07T03:00:00,defaulted,125990.0,2023-01-09 18:14:18.089674,15,default,no restruct,No auto-approve,2. silver,2023-01
...,...,...,...,...,...,...,...,...,...,...,...,...
67150,18046,22,2025-07-01T03:00:00,defaulted,36033.2,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver,2023-03
67151,18046,23,2025-08-01T03:00:00,defaulted,35977.0,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver,2023-03
67152,18046,24,2025-09-01T03:00:00,defaulted,35909.7,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver,2023-03
67153,18046,25,2025-10-01T03:00:00,defaulted,35837.8,2023-03-03 15:06:07.394743,13,default,restruct,No auto-approve,2. silver,2023-03


In [151]:
# Посчитаем сумму amount по фильтрам default
df3_M = df3.groupby(df3['new_month'])
Default_sum = df3_M['amount'].sum()
Default_sum

new_month
2023-01    21413052.17
2023-02    12726050.62
2023-03    26691040.84
2023-04    27413818.52
2023-05    22364916.00
2023-06    43563105.00
2023-07    24869366.04
2023-08    32724592.10
2023-09    17991990.20
2023-10    14615204.60
2023-11    21007406.20
2023-12     6298842.00
Freq: M, Name: amount, dtype: float64

In [152]:
# сделаем pandas dataframe чтобы построить график
default_sum_df = Default_sum.reset_index()
default_sum_df

,new_month,amount
0,2023-01,21413052.17
1,2023-02,12726050.62
2,2023-03,26691040.84
3,2023-04,27413818.52
4,2023-05,22364916.00
5,2023-06,43563105.00
6,2023-07,24869366.04
7,2023-08,32724592.10
8,2023-09,17991990.20
9,2023-10,14615204.60


In [153]:
#построим график
default_sum_df['new_month'] = default_sum_df['new_month'].astype(str)

fig = px.line(default_sum_df,
              x='new_month',
              y='amount',
              title='Когортный анализ дефолта по месяцам выдачи займа (start_date)')
fig.update_xaxes(title_text='Месяц')
fig.update_yaxes(title_text='Сумма займов')

fig.show()

# 4 - Напишите аналитический вывод на основании проделанной вами работы


### Выводы:
1. высокая доля NPL платежей в составе общих платежей связана с определёнными месяцами 2023 года (Март и Сентябрь). Нужно проверить, чем деятельность компании в это время отличалась от деятельности компании в остальное время (состав работников, акции компании и пр.), а так же обратить внимание внешние неподконтрольные факторы, такие как новости экономики и политики, изменения в законодательстве в смежных професииональных областях и пр.

2. Высокий уровень просроченных платежей и дефолтных займов может указывать на необходимость корректировки критериев кредитования или улучшения системы мониторинга клиентской базы.
3. с увеличением рейтинга займа растёт доля займов со статусом "defaulted" и "potentially defaulted"